Remarque : fait à l'arrache dans le notebook pour l'instant, déplacement dans un .py plus tard

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
import sys
import torchtext
sys.path.append("..")
from common import get_positional_encodings
import nltk

In [2]:
#entrée : positional encoding + embeddings
#ajouter layer norm

In [13]:
# Paramètres

nb_decoders = 6
nb_heads = 8
vector_size = 512
vocab_size = 1000
ffn_hidden_size = 100
seq_length = 15
attention_features_size = 5


In [20]:
class Transformer(nn.Module):
    def __init__(self, decoder):
        super(Transformer, self).__init__()
        self.decoder = decoder
        self.embedding = nn.Embedding(vocab_size, 4)
        self.finalfc = nn.Linear(vector_size, vocab_size)

    def forward(self, x):
        
        embedded = embedding(x)
        pos_encodings = torch.tensor(get_positional_encodings(seq_length, vector_size))
        x = embedded + pos_encodings
        
        for x in range(nb_decoders):
            x = self.decoder(x)
            
        x = F.softmax(self.finalfc(x))
        return x

In [12]:
class Decoder(nn.Module):
    "Decoder block applying multihead attention mechanism followed by a feedforward network"
    def __init__(self, multihead_attention, feedforward_network):
        super(Decoder, self).__init__()
        self.multihead_attention = multihead_attention
        self.feedforward_network = feedforward_network
        self.layernorm = nn.modules.normalization.LayerNorm

    def forward(self, x):
        #normalisation a la fin ou au debut ?
        mha = self.multihead_attention(self.layernorm(x))
        x += mha
        ffo = self.feedforward_network(self.layernorm(x))
        x += ffo
        return x
    

In [23]:
class FeedforwardNetwork(nn.Module):
    def __init__(self, vector_size, hidden_size):
        "Classic Feedforward Network with two layers"
        super(FeedforwardNetwork, self).__init__()
        self.hidden_size  = hidden_size
        self.fc1 = nn.Linear(vector_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, vector_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
ffn = FeedforwardNetwork(10,5)

In [24]:
class MultiHeadAttention(nn.Module):
    "MultiHead Attention Block"
    def __init__(self, nb_heads, features_size, vector_size):
        super(MultiHeadAttention, self).__init__()
        self.nb_heads = nb_heads
        self.features_size = features_size
        #A changer
        self.w_q = nn.Linear(vector_size, self.nb_heads*self.features_size)
        self.w_k = nn.Linear(vector_size, self.nb_heads*self.features_size)
        self.w_v = nn.Linear(vector_size, self.nb_heads*self.features_size)
        self.w_0 = nn.Linear(self.nb_heads*vector_size, vector_size)
        
    def mask_w(w):
        #pseudo code
        return w*(w.index() >= present)
    
    def attention(self, q, k, v):
        w = torch.matmul(q, k.transpose(1,0)) / math.sqrt() #v.shape[-1].value
        w = mask_w(w)
        w = F.softmax(w)
        a = tf.matmul(w, v)
        return a
        
    def forward(self):
        q = self.w_q(x).view(self.nb_heads, -1)
        k = self.w_k(x).view(self.nb_heads, -1)
        v = self.w_v(x).view(self.nb_heads, -1)
        a = attention(q, k, v) #redimensionner ?
        a = self.w_0(a)
        return a


In [27]:
LMtransformer = Transformer(Decoder(MultiHeadAttention(nb_heads, attention_features_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.001)

In [8]:
with open("fr.train.top1M.txt","r",encoding="utf8") as file:
    text = file.read()

a


In [ ]:
tokens = nltk.word_tokenize(text)

In [18]:
def train_model(nb_epochs, batch_size):
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        for i in range(batch_size):
            
            batch = tokens[i:i+batch_size]
            optimizer.zero_grad()
            outputs = LMtransformer(batch)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.